In [2]:
import numpy as np
import kaiwu as kw
kw.license.init(user_id="72317291601100802", sdk_code="vDSsMrcS1XvoHxrKEyWGPu3y6bydtx")
from kaiwu.classical import SimulatedAnnealingOptimizer

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# First Model

In [26]:
import numpy as np
import random
import math
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple CNN model for CIFAR-10
def build_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer for 10 classes
    return model

# Compile the model
model = build_cnn_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (for initial weights)
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Extract the trained weights (kernel and bias) from the first Conv2D layer
conv2d_layer = model.layers[0]  # Assuming the first Conv2D layer
kernel_weights, bias_weights = conv2d_layer.get_weights()  # Extract kernel and bias

# binarized_kernel_weights = np.sign(kernel_weights)  # Binarize the kernel weights to +1 or -1
# bias_weights will be left unchanged in this case
binarized_kernel_weights = kernel_weights


Epoch 1/5
782/782 [==============================] - 9s 11ms/step - loss: 1.5206 - accuracy: 0.4450 - val_loss: 1.3459 - val_accuracy: 0.5274
Epoch 2/5
782/782 [==============================] - 8s 11ms/step - loss: 1.1629 - accuracy: 0.5874 - val_loss: 1.1241 - val_accuracy: 0.5995
Epoch 3/5
782/782 [==============================] - 8s 11ms/step - loss: 0.9935 - accuracy: 0.6514 - val_loss: 1.0394 - val_accuracy: 0.6336
Epoch 4/5
782/782 [==============================] - 8s 11ms/step - loss: 0.8814 - accuracy: 0.6919 - val_loss: 0.9171 - val_accuracy: 0.6767
Epoch 5/5
782/782 [==============================] - 8s 11ms/step - loss: 0.7897 - accuracy: 0.7242 - val_loss: 0.8597 - val_accuracy: 0.6993


In [27]:
predictions = model.predict(x_test)  # x_test is the CIFAR-10 test data
predicted_labels = np.argmax(predictions, axis=1)  # Get the index with the highest probability for each sample
true_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predicted_labels == true_labels)  # Compare predicted vs. true labels

# Print the accuracy
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step
Test Set Accuracy: 69.93%


In [11]:
def generate_qubo_matrix(weights,idx):
    # num_weights = len(weights)
    num_weights = 600
    Q = np.zeros((num_weights, num_weights))
    
    # Penalize large weights and encourage sparsity
    for i in range(num_weights):
        Q[i, i] = abs(weights[i+idx])  # Larger weights are penalized more
    
    return Q


initial_weights = []
initial_biases = []
for layer in model.layers:
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
        kernel, bias = layer.get_weights()
        initial_weights.append(kernel.flatten())  # Flatten kernels (weights)
        initial_biases.append(bias.flatten())    # Flatten biases

# Concatenate the initial weights and biases
initial_weights = np.concatenate(initial_weights)
initial_biases = np.concatenate(initial_biases)

binary_weights_all = np.zeros(1)
for i in range(int(initial_weights.shape[0] / 600)):
    idx = i*600
    Q = Q = generate_qubo_matrix(initial_weights,idx)
    solver = SimulatedAnnealingOptimizer()

    # Solve the QUBO problem
    solution = solver.solve(Q)
    best_solution = solution[0]
    
    optimized_x = best_solution
    binary_weights = np.zeros(600)
    binary_weights[optimized_x == 1] = initial_weights[i*600:i*600+600][optimized_x == 1]
    binary_weights_all = np.concatenate((binary_weights_all, binary_weights))


In [14]:
binary_weights_all_new = binary_weights_all[1:]
binary_weights_all_new = np.concatenate((binary_weights_all_new, np.array([0] * (len(initial_weights) - len(binary_weights_all_new)))))

In [28]:
# Reconstruct weights and biases for each layer
layer_idx_w = 0
layer_idx_b = 0
updated_weights = []
updated_biases = []
for layer in model.layers:
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
        num_params = layer.get_weights()[0].size  # Kernel size
        num_biases = layer.get_weights()[1].size  # Bias size

        # Get the kernel weights for the layer
        layer_kernel = binary_weights_all_new[layer_idx_w:layer_idx_w + num_params].reshape(layer.get_weights()[0].shape)
        updated_weights.append(layer_kernel)

        # Get the biases for the layer
        # layer_bias = binary_weights_all_new[layer_idx + num_params:layer_idx + num_params + num_biases].reshape(layer.get_weights()[1].shape)
        layer_bias = initial_biases[layer_idx_b:layer_idx_b + num_biases].reshape(layer.get_weights()[1].shape)
        updated_biases.append(layer_bias)

        # Move to the next set of weights
        # layer_idx += num_params + num_biases
        layer_idx_w += num_params
        layer_idx_b += num_biases

# Now set the weights and biases to the model
updated_weights = [w for w in updated_weights]
updated_biases = [b for b in updated_biases]

# Set the new weights and biases for the model
model.set_weights([w for pair in zip(updated_weights, updated_biases) for w in pair])

# Verify the model summary after setting the weights
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_4 (Flatten)         (None, 2048)             

In [29]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

Epoch 1/5
782/782 [==============================] - 9s 11ms/step - loss: 1.0850 - accuracy: 0.6164 - val_loss: 0.9775 - val_accuracy: 0.6595
Epoch 2/5
782/782 [==============================] - 8s 11ms/step - loss: 0.8193 - accuracy: 0.7144 - val_loss: 0.8689 - val_accuracy: 0.6934
Epoch 3/5
782/782 [==============================] - 9s 11ms/step - loss: 0.7117 - accuracy: 0.7517 - val_loss: 0.8322 - val_accuracy: 0.7081
Epoch 4/5
782/782 [==============================] - 9s 11ms/step - loss: 0.6484 - accuracy: 0.7723 - val_loss: 0.8026 - val_accuracy: 0.7248
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.8128 - accuracy: 0.7248
Test accuracy: 72.48%


# Second Model

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define the deep learning model (a simple CNN)
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define a function to create a more complex QUBO matrix


def create_complex_qubo(weights, idx,sparsity_factor=0.1, interaction_factor=0.01, entropy_factor=0.1):
    # n_weights = len(weights)
    n_weights = 600
    Q = np.zeros((n_weights, n_weights))
    print(idx)
    
    # Add sparsity regularization (penalize large weights, encourage sparsity)
    for i in range(n_weights):
        Q[i, i] += sparsity_factor * weights[i+idx] ** 2  # Penalize non-zero weights
    
    # Add interaction regularization (encourage some weights to be correlated or anticorrelated)
    for i in range(n_weights):
        for j in range(i + 1, n_weights):
            Q[i, j] += interaction_factor * weights[i+idx] * weights[j+idx]  # Penalize uncorrelated weights
    
    # Add entropy minimization (encourage confident predictions)
    for i in range(n_weights):
        for j in range(i + 1, n_weights):
            Q[i, j] += entropy_factor * (weights[i+idx] - 0.5) * (weights[j+idx] - 0.5)  # Enforce weights to be either 0 or 1
    
    return Q

# Define the energy function for QUBO
def qubo_energy(x, Q):
    return np.dot(x.T, np.dot(Q, x))  # Calculate the energy (objective function)

# Initialize the model and get the weights
model = create_model()
initial_weights = []
initial_biases = []
for layer in model.layers:
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
        kernel, bias = layer.get_weights()
        initial_weights.append(kernel.flatten())  # Flatten kernels (weights)
        initial_biases.append(bias.flatten())    # Flatten biases

# Concatenate the initial weights and biases
initial_weights = np.concatenate(initial_weights)
initial_biases = np.concatenate(initial_biases)


# Create a more complex QUBO matrix
binary_weights_all = np.zeros(1)
for i in range(int(initial_weights.shape[0] / 600)):
    idx = i*600
    Q = create_complex_qubo(initial_weights,idx)
    solver = SimulatedAnnealingOptimizer()

    # Solve the QUBO problem
    solution = solver.solve(Q)
    best_solution = solution[0]
    
    optimized_x = best_solution
    binary_weights = np.zeros(600)
    binary_weights[optimized_x == 1] = initial_weights[i*600:i*600+600][optimized_x == 1]
    binary_weights_all = np.concatenate((binary_weights_all, binary_weights))

binary_weights_all_new = binary_weights_all[1:]
binary_weights_all_new = np.concatenate((binary_weights_all_new, np.array([0] * (len(initial_weights) - len(binary_weights_all_new)))))

0
600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600
7200
7800
8400
9000
9600
10200
10800
11400
12000
12600
13200
13800
14400
15000
15600
16200
16800
17400
18000
18600
19200
19800
20400
21000
21600
22200
22800
23400
24000
24600
25200
25800
26400
27000
27600
28200
28800
29400
30000
30600
31200
31800
32400
33000
33600
34200
34800
35400
36000
36600
37200
37800
38400
39000
39600
40200
40800
41400
42000
42600
43200
43800
44400
45000
45600
46200
46800
47400
48000
48600
49200
49800
50400
51000
51600
52200
52800
53400
54000
54600
55200
55800
56400
57000
57600
58200
58800
59400
60000
60600
61200
61800
62400
63000
63600
64200
64800
65400
66000
66600
67200
67800
68400
69000
69600
70200
70800
71400
72000
72600
73200
73800
74400
75000
75600
76200
76800
77400
78000
78600
79200
79800
80400
81000
81600
82200
82800
83400
84000
84600
85200
85800
86400
87000
87600
88200
88800
89400
90000
90600
91200
91800
92400
93000
93600
94200
94800
95400
96000
96600
97200
97800
98400
99000
99600
100200
100800
10140

In [7]:
# Reconstruct weights and biases for each layer
layer_idx_w = 0
layer_idx_b = 0
updated_weights = []
updated_biases = []
for layer in model.layers:
    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Dense):
        num_params = layer.get_weights()[0].size  # Kernel size
        num_biases = layer.get_weights()[1].size  # Bias size

        # Get the kernel weights for the layer
        layer_kernel = binary_weights_all_new[layer_idx_w:layer_idx_w + num_params].reshape(layer.get_weights()[0].shape)
        updated_weights.append(layer_kernel)

        # Get the biases for the layer
        # layer_bias = binary_weights_all_new[layer_idx + num_params:layer_idx + num_params + num_biases].reshape(layer.get_weights()[1].shape)
        layer_bias = initial_biases[layer_idx_b:layer_idx_b + num_biases].reshape(layer.get_weights()[1].shape)
        updated_biases.append(layer_bias)

        # Move to the next set of weights
        # layer_idx += num_params + num_biases
        layer_idx_w += num_params
        layer_idx_b += num_biases

# Now set the weights and biases to the model
updated_weights = [w for w in updated_weights]
updated_biases = [b for b in updated_biases]

# Set the new weights and biases for the model
model.set_weights([w for pair in zip(updated_weights, updated_biases) for w in pair])

# Verify the model summary after setting the weights
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [8]:
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 1.6383 - accuracy: 0.3953 - val_loss: 1.3699 - val_accuracy: 0.5031
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 1.3053 - accuracy: 0.5338 - val_loss: 1.2174 - val_accuracy: 0.5742
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 1.1414 - accuracy: 0.5979 - val_loss: 1.0696 - val_accuracy: 0.6296
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 1.0310 - accuracy: 0.6406 - val_loss: 1.0255 - val_accuracy: 0.6466
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.9503 - accuracy: 0.6676 - val_loss: 1.0283 - val_accuracy: 0.6442
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.8883 - accuracy: 0.6918 - val_loss: 0.9249 - val_accuracy: 0.6790
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.8272 - accuracy: 0.7117 - val_loss: 0.9096 - val_accuracy: 0.6871
Epoch 